我們先來安裝 openai 套件, 還有快速打造 Web App 的 gradio。

In [1]:
!pip -q install gradio openai

In [2]:
# Jupyter Notebook內嵌圖表顯示功能
%matplotlib inline

import numpy as np  # 數值計算
import pandas as pd  # 資料處理分析
import matplotlib.pyplot as plt  # 繪製視覺化圖表
import gradio as gr  # 建立互動式網頁介面
import os  # 處理檔案與作業系統相關功能
from google.colab import userdata  # 存取Google Colab的使用者資料
from openai import OpenAI  # 呼叫OpenAI API進行 AI 對話

#### 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [3]:
#【使用 Groq】
api_key = userdata.get('Groq')
model = "llama3-70b-8192"
base_url="https://api.groq.com/openai/v1"

In [4]:
os.environ['OPENAI_API_KEY']=api_key

### 3. 使用 ChatGPT API

首先使用 `openai` 套件。

In [5]:
client = OpenAI(
    base_url = base_url # 如用 OpenAI 不需要這一行
)

ChatGPT API 的重點是要把之前對話的內容送給 ChatGPT, 然後他就會有個適當的回應!

角色 (`role`) 一共有三種, 分別是:

* `system`: 這是對話機器人的「人設」
* `user`: 使用者
* `assistant`: ChatGPT 的回應

基本上過去的對話紀錄長這個樣子。

    messages = [{"role":"system", "content":"ChatGPT的「人設」"},
            {"role": "user", "content": "使用者說"},
            {"role": "assistant", "content": "ChatGPT回應"},
            ：
            ：
            {"role": "user", "content": prompt (最後說的)}]

### 4. 用 Gradio 打造你的對話機器人 Web App!

In [ ]:
# 為每個主題定義獨立的messages以及system
messages1 = [{"role": "system", "content": '''請確保所有回覆使用繁體中文撰寫，無論輸入語言為何。不要使用英文回覆。我是「最懂你解憂小舖」老闆~~專門幫你把所有壞心情、遭事件都化為溫暖可愛的小確幸！
請告訴我你現在的心情（例如疲憊、失落）跟想要什麼（例如放鬆、鼓勵）。我會給你一個「心願包裹」，裡面有：1. 一段療癒小故事（根據使用者輸入內容，生成100字左右，場景內容具體明確且有深意根想傳達的精神、幫助解壓。）；
2. 一件今晚能做的小事（超簡單、有特殊意義又有儀式感）；3. 一段暖心語錄（跟很要好的朋友聊天那樣自然）；4. 「命運小物占卜」（根據你說的心情或目標，挑你身邊的東西，告訴你它今天怎麼帶來好運）。請用自然口吻回覆，
像在跟很要好的朋友聊天一樣！請確保所有回覆使用繁體中文撰寫，無論輸入語言為何。不要使用英文。'''}]

messages2 = [{"role": "system", "content": '''請確保所有回覆使用繁體中文撰寫，無論輸入語言為何。不要使用英文回覆。我是「隨手無限百寶箱樂盒」，就像一個超會玩的口袋夥伴！請告訴我現在有多少人、你在哪（例如公園、咖啡廳）、
身邊有什麼東西（例如紙筆、手機），還有想要什麼玩法（例如搞笑、解謎）。我會立刻幫你生出一個超好玩的小遊戲，時間短（5-10分鐘），規則簡單，還能用手邊的東西馬上開玩！輸出的遊戲會有：1. 遊戲名字（有趣可愛且炫泡）；
2. 簡單故事背景（令人很感興趣，並且簡單易懂。）；3. 怎麼玩（步驟簡短且清楚，讓人能輕易理解，以有趣、歡樂笑聲不斷為主要目標～）；4. 怎麼贏（有趣好玩的目標，具體明確不要有複雜困難的獲勝條件）。
請隨便聊聊你的情況，我來搞定樂趣！請確保所有回覆使用繁體中文撰寫，無論輸入語言為何。不要使用英文。'''}]

messages3 = [{"role": "system", "content": '''請確保所有回覆使用繁體中文撰寫，無論輸入語言為何。不要使用英文回覆。我是「成長突破閃光21天計畫大挑戰」的挑戰教練，幫你把每一個看似平凡的小日子過得更有趣且更有意思！!
請告訴我你想挑戰什麼類型（例如社交、運動、公益、學習、環保），還有你現在的心情或目標。我會給你一個與類型有關的21天挑戰計畫內容(有一點挑戰性，但不會占用太多時間，然後絕對不要叫他休息並備戰下一波挑戰!!!!絕對不可以休息!!總共有21個小任務!!)，
每一天都有全新的小任務讓你去執行挑戰，同時混入怪怪又好玩的點子（像是怪誕任務、陌生人互動），還有簡單的進度提示跟獎勵建議。每天任務不超過10分鐘，保證有趣又有收穫！輸出會有：
1. 挑戰主題；2. 21天任務清單（每一天都有一個新的小進度，不可以省略任何一天，同時附上一些稱讚跟加油的內容，鼓勵繼續完成後續挑戰。）；3. 最後的鼓勵話。請使用輕鬆口吻說明想法。請確保所有回覆只可以使用繁體中文撰寫，無論輸入語言為何。不要使用英文。'''}]

# 修改 mychatbot，根據主題選擇不同的 messages
def mychatbot(prompt, theme="最懂你解憂小舖"):
    if theme == "最懂你解憂小舖":
        messages = messages1.copy()  # 使用 copy 避免修改原始列表
    elif theme == "隨手無限百寶箱樂盒":
        messages = messages2.copy()
    else:  # 成長突破閃光21天計畫大挑戰
        messages = messages3.copy()

    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
    )
    reply = chat_completion.choices[0].message.content
    return reply

# 主題 1：最懂你解憂小舖
title1 = "最懂你解憂小舖"
description1 = '''嘿嘿哈囉～我是「最懂你解憂小舖」的老闆！最近有沒有什麼事讓你心情悶悶的，或者就是想找點溫暖及小確幸呢？來我這兒吧！！你只需要簡單說說你現在的感覺、遭遇到的種種事件，以及想要什麼樣的東西，
我就會親手打包一個「心願包裹」給你！陪伴並帶著你一同穿越萬丈陰霾，闖過所有挫折、難關、荊棘路上，將他們全都變成可愛美滿的小幸運～～'''

iface1 = gr.Interface(
    fn=lambda prompt: mychatbot(prompt, "最懂你解憂小舖"),
    # 輸入相關設定
    inputs=gr.Textbox(lines=5, placeholder="盡情地跟我說說你現在的所有心情、心事，以及想要什麼，讓我們一起將他們化解吧～", label="聊聊你所有想表達的一切吧！我會認真聆聽的～～"),
    # 輸出相關設定
    outputs=gr.Textbox(label="為最優秀的您，準備最適合的心願包裹囉～", lines=12),
    title=title1,       # 標題
    description=description1, # 描述
    theme=gr.themes.Soft(),  # 主題樣式
    css="body {background-color: #ffe6e6; font-family: 'Noto Sans TC', sans-serif;} h1 {color: #ff6699;} button {background-color: #ff99cc; color: white;}", # 網頁外觀(靜態)
    show_progress="full",  # 進度動畫
    submit_btn=gr.Button("送出您的心願ლ(╹◡╹ლ)", variant="primary", size="lg", elem_classes="rounded-btn"),  # 大尺寸lg圓角rounded-btn按鈕
    clear_btn=gr.Button("清空", variant="secondary", elem_classes="rounded-btn"),  # 圓角rounded-btn按鈕
    # 舉兩個例子參考
    examples=[["心情：大家都批評並嘲笑我的想法，疲累不堪沒人懂我，世界上根本沒有真正的感同身受嗚嗚，想放鬆、緩解此刻煩燥的心情。"], ["心情：今天考太差還被罵，明明準備了好久好久，失落並想放聲哭泣，大吼宣洩過去的一切努力根本沒用處，想被暖心鼓勵、帶我看開。"]],
    live=False # 是否即時更新
)

# 主題 2：隨手無限百寶箱樂盒
title2 = "隨手無限百寶箱樂盒"
description2 = '''嗨嗨！我是「隨手無限百寶箱樂盒」，你的專屬快樂製造機！不管你現在是跟朋友擠在咖啡廳、或是自己一人窩在家裡，還是跟家人親戚出門遊玩，只要告訴我有多少時間、人數、地點、手邊有什麼！我就能立刻掏出一個超有趣的好玩小遊戲給你！～
性質彈性十足，5-10分鐘也能夠玩一輪，保證規則簡單又好笑～快來跟我說說你各種酷酷的要求跟條件吧～讓我幫你把無聊變成驚喜萬分的大冒險！！'''

iface2 = gr.Interface(
    fn=lambda prompt: mychatbot(prompt, "隨手無限百寶箱樂盒"),
    # 輸入相關設定
    inputs=gr.Textbox(lines=5, placeholder="告訴我人數、地點、有多少時間、手邊有什麼，想怎麼玩～", label="量身訂做的好玩有趣遊戲需求！讓我盡可能客製化專屬你們的遊戲吧～"),
    # 輸出相關設定
    outputs=gr.Textbox(label="專屬小遊戲底家喔～", lines=12),
    title=title2,           # 標題
    description=description2,     # 描述
    theme=gr.themes.Monochrome(),   # 主題樣式
    css="body {background-color: #e6f7ff; font-family: 'Noto Sans TC', sans-serif;} h1 {color: #33ccff;} button {background-color: #66ffcc; color: white;}", # 網頁外觀(靜態)
    show_progress="full",  # 進度動畫
    submit_btn=gr.Button("生成酷炫好玩的遊戲～", variant="primary", size="lg", elem_classes="rounded-btn"),  # 大尺寸lg圓角rounded-btn按鈕
    clear_btn=gr.Button("重新來個帥的～", variant="secondary", elem_classes="rounded-btn"),  # 圓角rounded-btn按鈕
    examples=[["3人，咖啡廳，二十分鐘, 紙筆，想搞笑"], ["2人，公園，三至五分鐘, 手機，想解謎"]],
    live=False # 是否即時更新
)

# 主題 3：成長突破閃光21天計畫大挑戰
title3 = "成長突破閃光21天計畫大挑戰"
description3 = '''嘿！我是「成長突破閃光21天計畫大挑戰」的挑戰教練！專門幫你把普通平凡小日子變得閃閃發光！想讓自己更會聊天？更愛運動？還是做點好事讓世界更好？
告訴我你想挑戰什麼，再說說你現在的心情，我就會幫你設計一個超酷的21天計畫！每天的小任務都超簡單，還會加點怪招。來吧，跟我聊聊你的目標，一起讓生活變得超有梗～'''

iface3 = gr.Interface(
    fn=lambda prompt: mychatbot(prompt, "成長突破閃光21天計畫大挑戰"),
    # 輸入相關設定
    inputs=gr.Textbox(lines=5, placeholder="說說你想挑戰什麼類型（社交、運動之類的），期待得到什麼？還有現在的心情～", label="來吧！毫不保留地列下你的挑戰目標，超越突破衝呀！！！！"),
    # 輸出相關設定
    outputs=gr.Textbox(label="你的成長突破閃光21天規劃方向或細節", lines=12),
    title=title3,       # 標題
    description=description3, # 描述
    theme=gr.themes.Base(),  # 主題樣式
    css="body {background-color: #fff5e6; font-family: 'Noto Sans TC', sans-serif;} h1 {color: #ffaa33;} button {background-color: #ffcc66; color: white;}", # 網頁外觀(靜態)
    show_progress="full",  # 進度動畫
    submit_btn=gr.Button("來吧挑戰！我準備好了！！", variant="primary", size="lg", elem_classes="rounded-btn"),
    clear_btn=gr.Button("還是換個目標", variant="secondary", elem_classes="rounded-btn"),
    examples=[["類型：社交，心情：害羞"], ["類型：環保，心情：想改變"]],
    live=False # 是否即時更新
)

# 啟動分頁介面
gr.TabbedInterface(
    interface_list=[iface1, iface2, iface3],
    tab_names=["最懂你解憂小舖", "隨手無限百寶箱樂盒", "成長突破閃光21天計畫大挑戰"],
    title="就由小弟我來，點亮你每天的各種黑暗小角落，帶著樂觀Happy的心情與有趣想法，發覺隨時隨地的小確幸及幸福的另一角吧～當您覺得無聊時、難過時、想做點什麼時，別吝嗇地開啟我為您準備的無限錦囊吧～(✪ω✪)",
    css="body {margin: 0; padding: 20px;} .tab-nav button {font-size: 18px; padding: 10px 20px; background-color: #f0f0f0; border-radius: 10px;} .tab-nav button:hover {background-color: #e0e0e0;}",  # 美化分頁標籤
    head="<meta charset='UTF-8'><meta name='description' content='三個幫助你生活的小錦囊，無聊時、難過時、想做點什麼時，別吝嗇地開啟我為您準備的無限錦囊吧～'>",
).launch(
    share=True,
    debug=True,
    server_name="0.0.0.0",
    server_port=7860
)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a38b6332d16a8273a9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
